In [ ]:
import os
import numpy as np
from skimage import io, segmentation
from PIL import Image

# ─── USER PARAMETERS ────────────────────────────────────────────────────────────
BASE_DIR      = '/content/drive/MyDrive/biotech/Retina_Lab/Image_data/cropped_prs'
OUT_DIR       = os.path.join(BASE_DIR, 'extracted_prs')
IMAGE_EXT     = '.png'
SEG_EXT       = '.npy'
NAME_TEMPLATE = 'pr_{:03d}.png' # define the naming template
OVERLAY_TMPL  = '{}_overlay.png'
# ────────────────────────────────────────────────────────────────────────────────

os.makedirs(OUT_DIR, exist_ok=True)

# 1) List what we actually see
all_files = sorted(os.listdir(BASE_DIR))
print("Files in folder:")
for f in all_files:
    print("  ", f)
print()

# 2) Pick out the images
images = [f for f in all_files if f.endswith(IMAGE_EXT)]
print(f"Found {len(images)} image(s) ending in {IMAGE_EXT}:")
for im in images:
    print("  ", im)
print()

cell_counter = 0

for img_fname in images:
    base, _ = os.path.splitext(img_fname)
    img_path = os.path.join(BASE_DIR, img_fname)
    seg_path = os.path.join(BASE_DIR, base + SEG_EXT)

    # 3) Check for matching .npy
    if not os.path.isfile(seg_path):
        print(f"Skipping {img_fname}: no matching {SEG_EXT} found at {seg_path}")
        continue
    print(f"Processing {img_fname} with {os.path.basename(seg_path)}")

    # 4) Load and inspect the .npy
    raw = np.load(seg_path, allow_pickle=True)
    if isinstance(raw, np.ndarray) and raw.dtype == 'object':
        data = raw.item()
    else:
        data = raw
    print("  raw type:", type(raw))
    if hasattr(raw, 'files'):
        print("  npz keys:", raw.files)
    if isinstance(data, dict):
        print("  dict keys:", list(data.keys()))
    masks = data.get('masks', None)
    if masks is None:
        raise KeyError(f"Could not find key 'masks' in {seg_path}")

    unique = np.unique(masks)
    print("  unique labels in mask:", unique if unique.size <= 10 else unique[:10], "...")

    # 5) Save overlay
    img = io.imread(img_path)
    overlay = segmentation.mark_boundaries(img, masks, color=(1, 0, 0))
    over_name = OVERLAY_TMPL.format(base)
    over_path = os.path.join(OUT_DIR, over_name)
    io.imsave(over_path, (overlay * 255).astype('uint8'))
    print(f"  wrote overlay -> {over_name}")

    # 6) Extract cells
    for lab in unique:
        if lab == 0:
            continue
        cell_counter += 1
        m = masks == lab
        ys, xs = np.where(m)
        y0, x0 = ys.min(), xs.min()
        y1, x1 = ys.max() + 1, xs.max() + 1

        patch = img[y0:y1, x0:x1].copy()
        m_crop = m[y0:y1, x0:x1]
        if patch.ndim == 3:
            patch[~m_crop] = 0
        else:
            patch = patch * m_crop

        out_name = NAME_TEMPLATE.format(cell_counter)
        Image.fromarray(patch).save(os.path.join(OUT_DIR, out_name))

    print(f"  extracted {len(unique) - 1} cells from this image\n")

print(f"Done — total cells saved: {cell_counter}")


Files in folder:
   extracted_prs

Found 0 image(s) ending in .png:

Done — total cells saved: 0
